<a href="https://colab.research.google.com/github/james-caldwell1981/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/module2-make-features/LS_DS_112_Make_Features_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200> 

# Assignment:

- Replicate the lesson code.

 - This means that if you haven't followed along already, type out the things that we did in class. Forcing your fingers to hit each key will help you internalize the syntax of what we're doing. Make sure you understand each line of code that you're writing, google things that you don't fully understand.
 - [Lambda Learning Method for DS - By Ryan Herr](https://docs.google.com/document/d/1ubOw9B3Hfip27hF2ZFnW3a3z9xAgrUDRReOEo-FHCVs/edit?usp=sharing)
- Convert the `term` column from string to integer.
- Make a column named `loan_status_is_great`. It should contain the integer 1 if `loan_status` is "Current" or "Fully Paid." Else it should contain the integer 0.
- Make `last_pymnt_d_month` and `last_pymnt_d_year` columns.

In [50]:
import pandas as pd

#Bash command to get data in zip file
!wget https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip

--2020-06-02 20:27:57--  https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip
Resolving resources.lendingclub.com (resources.lendingclub.com)... 35.163.140.110, 52.89.160.109, 44.228.179.66
Connecting to resources.lendingclub.com (resources.lendingclub.com)|35.163.140.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘LoanStats_2018Q4.csv.zip.1’

LoanStats_2018Q4.cs     [              <=>   ]  22.10M  2.04MB/s    in 11s     

2020-06-02 20:28:10 (1.99 MB/s) - ‘LoanStats_2018Q4.csv.zip.1’ saved [23170772]



In [51]:
#Bash command to unzip data
!unzip LoanStats_2018Q4.csv.zip

Archive:  LoanStats_2018Q4.csv.zip
replace LoanStats_2018Q4.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [52]:
#Bash command to see the head of the raw file
!head LoanStats_2018Q4.csv

Notes offered by Prospectus (https://www.lendingclub.com/info/prospectus.action)
"id","member_id","loan_amnt","funded_amnt","funded_amnt_inv","term","int_rate","installment","grade","sub_grade","emp_title","emp_length","home_ownership","annual_inc","verification_status","issue_d","loan_status","pymnt_plan","url","desc","purpose","title","zip_code","addr_state","dti","delinq_2yrs","earliest_cr_line","inq_last_6mths","mths_since_last_delinq","mths_since_last_record","open_acc","pub_rec","revol_bal","revol_util","total_acc","initial_list_status","out_prncp","out_prncp_inv","total_pymnt","total_pymnt_inv","total_rec_prncp","total_rec_int","total_rec_late_fee","recoveries","collection_recovery_fee","last_pymnt_d","last_pymnt_amnt","next_pymnt_d","last_credit_pull_d","collections_12_mths_ex_med","mths_since_last_major_derog","policy_code","application_type","annual_inc_joint","dti_joint","verification_status_joint","acc_now_delinq","tot_coll_amt","tot_cur_bal","open_acc_6m","open_act_il","op

In [53]:
#Bash command to see the end of the raw file
!tail LoanStats_2018Q4.csv

"","","5600","5600","5600"," 36 months"," 13.56%","190.21","C","C1","","n/a","RENT","15600","Not Verified","Oct-2018","Current","n","","","credit_card","Credit card refinancing","836xx","ID","15.31","0","Aug-2012","0","","97","9","1","5996","34.5%","11","w","2926.62","2926.62","3603.44","3603.44","2673.38","930.06","0.0","0.0","0.0","May-2020","190.21","Jun-2020","May-2020","0","","1","Individual","","","","0","0","5996","0","0","0","1","20","0","","0","2","3017","35","17400","1","0","0","3","750","4689","45.5","0","0","20","73","13","13","0","13","","20","","0","3","5","4","4","1","9","10","5","9","0","0","0","0","100","25","1","0","17400","5996","8600","0","","","","","","","","","","","","N","","","","","","","","","","","","","","","N","","","","","",""
"","","23000","23000","23000"," 36 months"," 15.02%","797.53","C","C3","Tax Consultant","10+ years","MORTGAGE","75000","Source Verified","Oct-2018","Charged Off","n","","","debt_consolidation","Debt consolidation","352xx","AL","20.9

In [54]:
#read file into dataframe getting headers from 2nd row
df = pd.read_csv('LoanStats_2018Q4.csv', header=1)

df.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,...,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,20000.0,20000.0,20000.0,36 months,14.47%,688.13,C,C2,bus driver,4 years,OWN,52000.0,Source Verified,Dec-2018,Charged Off,n,NaN,NaN,debt_consolidation,Debt consolidation,681xx,NE,30.65,1.0,Jun-1979,2.0,15.0,NaN,6.0,0.0,15048.0,73%,22.0,w,0.00,0.00,8169.130000,8169.13,5731.54,2437.59,0.0,0.0,0.0,Feb-2020,0.00,NaN,Mar-2020,0.0,...,8.0,4.0,14.0,4.0,6.0,NaN,0.0,0.0,2.0,95.0,50.0,0.0,0.0,61699.0,33157.0,20700.0,40999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,688.13,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,688.13,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,16000.0,16000.0,16000.0,36 months,14.47%,550.51,C,C2,Carpenter,10+ years,RENT,150000.0,Source Verified,Dec-2018,Current,n,NaN,NaN,other,Other,917xx,CA,17.94,0.0,Nov-2001,1.0,NaN,NaN,24.0,0.0,43721.0,59.2%,29.0,w,9726.53,9726.53,8788.870000,8788.87,6273.47,2515.40,0.0,0.0,0.0,May-2020,0.00,Jun-2020,May-2020,0.0,...,4.0,22.0,25.0,19.0,24.0,0.0,0.0,0.0,0.0,100.0,85.7,0.0,0.0,116647.0,57465.0,32100.0,42847.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,8425.0,8425.0,8425.0,36 months,27.27%,345.18,E,E5,Senior Director Risk Management,3 years,MORTGAGE,450000.0,Verified,Dec-2018,Fully Paid,n,NaN,NaN,credit_card,Credit card refinancing,020xx,MA,12.37,0.0,Oct-1997,0.0,NaN,NaN,21.0,0.0,36812.0,65.7%,37.0,w,0.00,0.00,8735.148975,8735.15,8425.00,310.15,0.0,0.0,0.0,Feb-2019,8754.30,NaN,Mar-2020,0.0,...,16.0,13.0,17.0,8.0,21.0,0.0,0.0,0.0,2.0,100.0,50.0,0.0,0.0,768304.0,189194.0,45800.0,189054.0,141905.0,Jun-2013,0.0,1.0,4.0,77.4,1.0,4.0,0.0,0.0,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,10000.0,10000.0,10000.0,60 months,12.98%,227.43,B,B5,Printer,9 years,RENT,60000.0,Not Verified,Dec-2018,Current,n,NaN,NaN,credit_card,Credit card refinancing,660xx,KS,14.90,0.0,May-2007,0.0,NaN,112.0,7.0,1.0,10677.0,54.2%,12.0,w,7787.21,7787.21,3805.020000,3805.02,2212.79,1592.23,0.0,0.0,0.0,May-2020,227.43,Jun-2020,May-2020,0.0,...,6.0,5.0,6.0,5.0,7.0,0.0,0.0,0.0,1.0,100.0,33.3,1.0,0.0,36200.0,27595.0,13000.0,16500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,30000.0,30000.0,30000.0,60 months,26.31%,903.73,E,E4,Casino Host,10+ years,OWN,62000.0,Source Verified,Dec-2018,Charged Off,n,

In [55]:
#Skip first row for headers instead of referencing row
df = pd.read_csv('LoanStats_2018Q4.csv', skiprows=1)

print(df.shape)
df.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(128414, 144)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,...,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,20000.0,20000.0,20000.0,36 months,14.47%,688.13,C,C2,bus driver,4 years,OWN,52000.0,Source Verified,Dec-2018,Charged Off,n,NaN,NaN,debt_consolidation,Debt consolidation,681xx,NE,30.65,1.0,Jun-1979,2.0,15.0,NaN,6.0,0.0,15048.0,73%,22.0,w,0.00,0.00,8169.130000,8169.13,5731.54,2437.59,0.0,0.0,0.0,Feb-2020,0.00,NaN,Mar-2020,0.0,...,8.0,4.0,14.0,4.0,6.0,NaN,0.0,0.0,2.0,95.0,50.0,0.0,0.0,61699.0,33157.0,20700.0,40999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,688.13,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,688.13,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,16000.0,16000.0,16000.0,36 months,14.47%,550.51,C,C2,Carpenter,10+ years,RENT,150000.0,Source Verified,Dec-2018,Current,n,NaN,NaN,other,Other,917xx,CA,17.94,0.0,Nov-2001,1.0,NaN,NaN,24.0,0.0,43721.0,59.2%,29.0,w,9726.53,9726.53,8788.870000,8788.87,6273.47,2515.40,0.0,0.0,0.0,May-2020,0.00,Jun-2020,May-2020,0.0,...,4.0,22.0,25.0,19.0,24.0,0.0,0.0,0.0,0.0,100.0,85.7,0.0,0.0,116647.0,57465.0,32100.0,42847.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,8425.0,8425.0,8425.0,36 months,27.27%,345.18,E,E5,Senior Director Risk Management,3 years,MORTGAGE,450000.0,Verified,Dec-2018,Fully Paid,n,NaN,NaN,credit_card,Credit card refinancing,020xx,MA,12.37,0.0,Oct-1997,0.0,NaN,NaN,21.0,0.0,36812.0,65.7%,37.0,w,0.00,0.00,8735.148975,8735.15,8425.00,310.15,0.0,0.0,0.0,Feb-2019,8754.30,NaN,Mar-2020,0.0,...,16.0,13.0,17.0,8.0,21.0,0.0,0.0,0.0,2.0,100.0,50.0,0.0,0.0,768304.0,189194.0,45800.0,189054.0,141905.0,Jun-2013,0.0,1.0,4.0,77.4,1.0,4.0,0.0,0.0,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,10000.0,10000.0,10000.0,60 months,12.98%,227.43,B,B5,Printer,9 years,RENT,60000.0,Not Verified,Dec-2018,Current,n,NaN,NaN,credit_card,Credit card refinancing,660xx,KS,14.90,0.0,May-2007,0.0,NaN,112.0,7.0,1.0,10677.0,54.2%,12.0,w,7787.21,7787.21,3805.020000,3805.02,2212.79,1592.23,0.0,0.0,0.0,May-2020,227.43,Jun-2020,May-2020,0.0,...,6.0,5.0,6.0,5.0,7.0,0.0,0.0,0.0,1.0,100.0,33.3,1.0,0.0,36200.0,27595.0,13000.0,16500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,30000.0,30000.0,30000.0,60 months,26.31%,903.73,E,E4,Casino Host,10+ years,OWN,62000.0,Source Verified,Dec-2018,Charged Off,n,

In [56]:
#ad up null values by column and sort least to greatest

df.isnull().sum().sort_values()

inq_fi                                             2
mo_sin_old_rev_tl_op                               2
delinq_amnt                                        2
chargeoff_within_12_mths                           2
acc_open_past_24mths                               2
inq_last_12m                                       2
total_cu_tl                                        2
total_rev_hi_lim                                   2
open_rv_24m                                        2
open_rv_12m                                        2
total_bal_il                                       2
open_il_24m                                        2
open_il_12m                                        2
open_act_il                                        2
open_acc_6m                                        2
tot_cur_bal                                        2
tot_coll_amt                                       2
acc_now_delinq                                     2
application_type                              

In [57]:
#skip footer which is causing NaNs

df = pd.read_csv('LoanStats_2018Q4.csv', skiprows=1, skipfooter=2, engine='python')

print(df.shape)
df.head()

(128412, 144)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,...,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,20000,20000,20000.0,36 months,14.47%,688.13,C,C2,bus driver,4 years,OWN,52000.0,Source Verified,Dec-2018,Charged Off,n,NaN,NaN,debt_consolidation,Debt consolidation,681xx,NE,30.65,1,Jun-1979,2,15.0,NaN,6,0,15048,73%,22,w,0.00,0.00,8169.130000,8169.13,5731.54,2437.59,0.0,0.0,0.0,Feb-2020,0.00,NaN,Mar-2020,0,...,8,4,14,4,6,NaN,0,0,2,95.0,50.0,0,0,61699,33157,20700,40999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,688.13,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,688.13,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,16000,16000,16000.0,36 months,14.47%,550.51,C,C2,Carpenter,10+ years,RENT,150000.0,Source Verified,Dec-2018,Current,n,NaN,NaN,other,Other,917xx,CA,17.94,0,Nov-2001,1,NaN,NaN,24,0,43721,59.2%,29,w,9726.53,9726.53,8788.870000,8788.87,6273.47,2515.40,0.0,0.0,0.0,May-2020,0.00,Jun-2020,May-2020,0,...,4,22,25,19,24,0.0,0,0,0,100.0,85.7,0,0,116647,57465,32100,42847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,8425,8425,8425.0,36 months,27.27%,345.18,E,E5,Senior Director Risk Management,3 years,MORTGAGE,450000.0,Verified,Dec-2018,Fully Paid,n,NaN,NaN,credit_card,Credit card refinancing,020xx,MA,12.37,0,Oct-1997,0,NaN,NaN,21,0,36812,65.7%,37,w,0.00,0.00,8735.148975,8735.15,8425.00,310.15,0.0,0.0,0.0,Feb-2019,8754.30,NaN,Mar-2020,0,...,16,13,17,8,21,0.0,0,0,2,100.0,50.0,0,0,768304,189194,45800,189054,141905.0,Jun-2013,0.0,1.0,4.0,77.4,1.0,4.0,0.0,0.0,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,10000,10000,10000.0,60 months,12.98%,227.43,B,B5,Printer,9 years,RENT,60000.0,Not Verified,Dec-2018,Current,n,NaN,NaN,credit_card,Credit card refinancing,660xx,KS,14.90,0,May-2007,0,NaN,112.0,7,1,10677,54.2%,12,w,7787.21,7787.21,3805.020000,3805.02,2212.79,1592.23,0.0,0.0,0.0,May-2020,227.43,Jun-2020,May-2020,0,...,6,5,6,5,7,0.0,0,0,1,100.0,33.3,1,0,36200,27595,13000,16500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,30000,30000,30000.0,60 months,26.31%,903.73,E,E4,Casino Host,10+ years,OWN,62000.0,Source Verified,Dec-2018,Charged Off,n,NaN,NaN,debt_consolidation,Debt consolidation,920xx,CA,37.24,0,Oct-2007,0,NaN,86.0,15,1,18890,39.8%,36,w,0.00,0.00,2645.410000,2645.41,754.24,1891.17,0.0,0.0,0.0,Apr-2019,903.73,NaN,May-20

In [58]:
#add up all null values and list by column least to most
df.isnull().sum().sort_values()


inq_fi                                             0
mo_sin_old_rev_tl_op                               0
delinq_amnt                                        0
chargeoff_within_12_mths                           0
acc_open_past_24mths                               0
inq_last_12m                                       0
total_cu_tl                                        0
total_rev_hi_lim                                   0
open_rv_24m                                        0
open_rv_12m                                        0
total_bal_il                                       0
open_il_24m                                        0
open_il_12m                                        0
open_act_il                                        0
open_acc_6m                                        0
tot_cur_bal                                        0
tot_coll_amt                                       0
acc_now_delinq                                     0
application_type                              

In [59]:
#drop columns with zero data
df = df.drop(['desc', 'url', 'member_id', 'id'], axis=1)
df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,...,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,20000,20000,20000.0,36 months,14.47%,688.13,C,C2,bus driver,4 years,OWN,52000.0,Source Verified,Dec-2018,Charged Off,n,debt_consolidation,Debt consolidation,681xx,NE,30.65,1,Jun-1979,2,15.0,NaN,6,0,15048,73%,22,w,0.00,0.00,8169.130000,8169.13,5731.54,2437.59,0.0,0.0,0.0,Feb-2020,0.00,NaN,Mar-2020,0,NaN,1,Individual,NaN,...,8,4,14,4,6,NaN,0,0,2,95.0,50.0,0,0,61699,33157,20700,40999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,688.13,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,688.13,N,NaN,NaN,NaN,NaN,NaN,NaN
1,16000,16000,16000.0,36 months,14.47%,550.51,C,C2,Carpenter,10+ years,RENT,150000.0,Source Verified,Dec-2018,Current,n,other,Other,917xx,CA,17.94,0,Nov-2001,1,NaN,NaN,24,0,43721,59.2%,29,w,9726.53,9726.53,8788.870000,8788.87,6273.47,2515.40,0.0,0.0,0.0,May-2020,0.00,Jun-2020,May-2020,0,NaN,1,Individual,NaN,...,4,22,25,19,24,0.0,0,0,0,100.0,85.7,0,0,116647,57465,32100,42847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,8425,8425,8425.0,36 months,27.27%,345.18,E,E5,Senior Director Risk Management,3 years,MORTGAGE,450000.0,Verified,Dec-2018,Fully Paid,n,credit_card,Credit card refinancing,020xx,MA,12.37,0,Oct-1997,0,NaN,NaN,21,0,36812,65.7%,37,w,0.00,0.00,8735.148975,8735.15,8425.00,310.15,0.0,0.0,0.0,Feb-2019,8754.30,NaN,Mar-2020,0,NaN,1,Joint App,515000.0,...,16,13,17,8,21,0.0,0,0,2,100.0,50.0,0,0,768304,189194,45800,189054,141905.0,Jun-2013,0.0,1.0,4.0,77.4,1.0,4.0,0.0,0.0,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
3,10000,10000,10000.0,60 months,12.98%,227.43,B,B5,Printer,9 years,RENT,60000.0,Not Verified,Dec-2018,Current,n,credit_card,Credit card refinancing,660xx,KS,14.90,0,May-2007,0,NaN,112.0,7,1,10677,54.2%,12,w,7787.21,7787.21,3805.020000,3805.02,2212.79,1592.23,0.0,0.0,0.0,May-2020,227.43,Jun-2020,May-2020,0,NaN,1,Individual,NaN,...,6,5,6,5,7,0.0,0,0,1,100.0,33.3,1,0,36200,27595,13000,16500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
4,30000,30000,30000.0,60 months,26.31%,903.73,E,E4,Casino Host,10+ years,OWN,62000.0,Source Verified,Dec-2018,Charged Off,n,debt_consolidation,Debt consolidation,920xx,CA,37.24,0,Oct-2007,0,NaN,86.0,15,1,18890,39.8%,36,w,0.00,0.00,2645.410000,2

In [60]:
#data type of each column
df.dtypes

loan_amnt                                       int64
funded_amnt                                     int64
funded_amnt_inv                               float64
term                                           object
int_rate                                       object
installment                                   float64
grade                                          object
sub_grade                                      object
emp_title                                      object
emp_length                                     object
home_ownership                                 object
annual_inc                                    float64
verification_status                            object
issue_d                                        object
loan_status                                    object
pymnt_plan                                     object
purpose                                        object
title                                          object
zip_code                    

In [61]:
#look at the interest rate column
df['int_rate'].head(10)

0     14.47%
1     14.47%
2     27.27%
3     12.98%
4     26.31%
5     17.97%
6     11.80%
7     10.72%
8     16.14%
9     12.98%
Name: int_rate, dtype: object

In [62]:
#extract one record from interest rate column to examine characters
df['int_rate'][3]

' 12.98%'

In [63]:
'''
use dummy interest rate to model all issues
encountered in the interest rate column and
strip leading/trailing whitespace
'''
int_rate = ' 12.98%'

int_rate.strip()

'12.98%'

In [64]:
#strip trailing whitespace and then the % sign
int_rate.strip().strip('%')

'12.98'

In [65]:
#typecast to float from string
float(int_rate.strip().strip('%'))

12.98

In [66]:
#typecast and check datatype
type(float(int_rate.strip().strip('%')))

float

In [0]:
#create function to perform the above
def int_rate_to_float(int_rate):
    return float(int_rate.strip().strip('%'))

In [68]:
#call function to ensure proper results
int_rate_to_float(int_rate)

12.98

In [69]:
#verify typecast execution
type(int_rate_to_float(int_rate))

float

In [70]:
'''
prime list to hold values to be added
to new column 'int_rate_cleaned'
'''
cleaned = []

for int_rate in df['int_rate']:
    cleaned.append(int_rate_to_float(int_rate))

df['int_rate_cleaned'] = pd.Series(cleaned)

df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,...,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,int_rate_cleaned
0,20000,20000,20000.0,36 months,14.47%,688.13,C,C2,bus driver,4 years,OWN,52000.0,Source Verified,Dec-2018,Charged Off,n,debt_consolidation,Debt consolidation,681xx,NE,30.65,1,Jun-1979,2,15.0,NaN,6,0,15048,73%,22,w,0.00,0.00,8169.130000,8169.13,5731.54,2437.59,0.0,0.0,0.0,Feb-2020,0.00,NaN,Mar-2020,0,NaN,1,Individual,NaN,...,4,14,4,6,NaN,0,0,2,95.0,50.0,0,0,61699,33157,20700,40999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,688.13,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,688.13,N,NaN,NaN,NaN,NaN,NaN,NaN,14.47
1,16000,16000,16000.0,36 months,14.47%,550.51,C,C2,Carpenter,10+ years,RENT,150000.0,Source Verified,Dec-2018,Current,n,other,Other,917xx,CA,17.94,0,Nov-2001,1,NaN,NaN,24,0,43721,59.2%,29,w,9726.53,9726.53,8788.870000,8788.87,6273.47,2515.40,0.0,0.0,0.0,May-2020,0.00,Jun-2020,May-2020,0,NaN,1,Individual,NaN,...,22,25,19,24,0.0,0,0,0,100.0,85.7,0,0,116647,57465,32100,42847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,14.47
2,8425,8425,8425.0,36 months,27.27%,345.18,E,E5,Senior Director Risk Management,3 years,MORTGAGE,450000.0,Verified,Dec-2018,Fully Paid,n,credit_card,Credit card refinancing,020xx,MA,12.37,0,Oct-1997,0,NaN,NaN,21,0,36812,65.7%,37,w,0.00,0.00,8735.148975,8735.15,8425.00,310.15,0.0,0.0,0.0,Feb-2019,8754.30,NaN,Mar-2020,0,NaN,1,Joint App,515000.0,...,13,17,8,21,0.0,0,0,2,100.0,50.0,0,0,768304,189194,45800,189054,141905.0,Jun-2013,0.0,1.0,4.0,77.4,1.0,4.0,0.0,0.0,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,27.27
3,10000,10000,10000.0,60 months,12.98%,227.43,B,B5,Printer,9 years,RENT,60000.0,Not Verified,Dec-2018,Current,n,credit_card,Credit card refinancing,660xx,KS,14.90,0,May-2007,0,NaN,112.0,7,1,10677,54.2%,12,w,7787.21,7787.21,3805.020000,3805.02,2212.79,1592.23,0.0,0.0,0.0,May-2020,227.43,Jun-2020,May-2020,0,NaN,1,Individual,NaN,...,5,6,5,7,0.0,0,0,1,100.0,33.3,1,0,36200,27595,13000,16500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,12.98
4,30000,30000,30000.0,60 months,26.31%,903.73,E,E4,Casino Host,10+ years,OWN,62000.0,Source Verified,Dec-2018,Charged Off,n,debt_consolidation,Debt consolidation,920xx,CA,37.24,0,Oct-2007,0,NaN,86.0,15,1,18890,39.8%,36,w,0

In [71]:
#show last 5 column datatypes
df.dtypes[-5:]

settlement_date           object
settlement_amount        float64
settlement_percentage    float64
settlement_term          float64
int_rate_cleaned         float64
dtype: object

In [72]:
'''
convert all interest rates to floats after
stripping whitespace and % signs while
over-writing original column
'''
df['int_rate'] = df['int_rate'].apply(int_rate_to_float)
df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,...,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,int_rate_cleaned
0,20000,20000,20000.0,36 months,14.47,688.13,C,C2,bus driver,4 years,OWN,52000.0,Source Verified,Dec-2018,Charged Off,n,debt_consolidation,Debt consolidation,681xx,NE,30.65,1,Jun-1979,2,15.0,NaN,6,0,15048,73%,22,w,0.00,0.00,8169.130000,8169.13,5731.54,2437.59,0.0,0.0,0.0,Feb-2020,0.00,NaN,Mar-2020,0,NaN,1,Individual,NaN,...,4,14,4,6,NaN,0,0,2,95.0,50.0,0,0,61699,33157,20700,40999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,688.13,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,688.13,N,NaN,NaN,NaN,NaN,NaN,NaN,14.47
1,16000,16000,16000.0,36 months,14.47,550.51,C,C2,Carpenter,10+ years,RENT,150000.0,Source Verified,Dec-2018,Current,n,other,Other,917xx,CA,17.94,0,Nov-2001,1,NaN,NaN,24,0,43721,59.2%,29,w,9726.53,9726.53,8788.870000,8788.87,6273.47,2515.40,0.0,0.0,0.0,May-2020,0.00,Jun-2020,May-2020,0,NaN,1,Individual,NaN,...,22,25,19,24,0.0,0,0,0,100.0,85.7,0,0,116647,57465,32100,42847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,14.47
2,8425,8425,8425.0,36 months,27.27,345.18,E,E5,Senior Director Risk Management,3 years,MORTGAGE,450000.0,Verified,Dec-2018,Fully Paid,n,credit_card,Credit card refinancing,020xx,MA,12.37,0,Oct-1997,0,NaN,NaN,21,0,36812,65.7%,37,w,0.00,0.00,8735.148975,8735.15,8425.00,310.15,0.0,0.0,0.0,Feb-2019,8754.30,NaN,Mar-2020,0,NaN,1,Joint App,515000.0,...,13,17,8,21,0.0,0,0,2,100.0,50.0,0,0,768304,189194,45800,189054,141905.0,Jun-2013,0.0,1.0,4.0,77.4,1.0,4.0,0.0,0.0,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,27.27
3,10000,10000,10000.0,60 months,12.98,227.43,B,B5,Printer,9 years,RENT,60000.0,Not Verified,Dec-2018,Current,n,credit_card,Credit card refinancing,660xx,KS,14.90,0,May-2007,0,NaN,112.0,7,1,10677,54.2%,12,w,7787.21,7787.21,3805.020000,3805.02,2212.79,1592.23,0.0,0.0,0.0,May-2020,227.43,Jun-2020,May-2020,0,NaN,1,Individual,NaN,...,5,6,5,7,0.0,0,0,1,100.0,33.3,1,0,36200,27595,13000,16500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,12.98
4,30000,30000,30000.0,60 months,26.31,903.73,E,E4,Casino Host,10+ years,OWN,62000.0,Source Verified,Dec-2018,Charged Off,n,debt_consolidation,Debt consolidation,920xx,CA,37.24,0,Oct-2007,0,NaN,86.0,15,1,18890,39.8%,36,w,0.00,0

In [73]:
#display top 20 most common entries with their occurence number
df['emp_title'].value_counts(dropna=False).head(20)

NaN                   20947
Teacher                2090
Manager                1773
Registered Nurse        952
Driver                  924
RN                      726
Supervisor              697
Sales                   580
Project Manager         526
General Manager         523
Office Manager          521
Owner                   420
Director                402
Truck Driver            387
Operations Manager      387
Nurse                   326
Engineer                325
Sales Manager           304
manager                 301
Supervisor              270
Name: emp_title, dtype: int64

In [74]:
#check for number of unique entries in 'emp_title' column
df['emp_title'].nunique()

43892

In [75]:
#check number of null values in 'emp_title' column
df['emp_title'].isnull().sum()

20947

In [81]:
import numpy as np

'''
I'm not totally certain how this works. My understanding
is that we are checking employee job titles against
the example list. If it is in the list, we strip
whitespace and save the cleaned version to the cell.
If it is not in the list, we save 'Unknown' to the cell.
'''

examples = ['owner', 'Supervisor', ' Project Manager', np.NaN]

def clean_emp_title(title):
    if isinstance(title, str):
        return title.strip().title()
    else:
        return "Unknown"

for example in examples:
    print(clean_emp_title(example))

Owner
Supervisor
Project Manager
Unknown


In [80]:
df['emp_title'].head()

0                         bus driver
1                          Carpenter
2    Senior Director Risk Management
3                            Printer
4                        Casino Host
Name: emp_title, dtype: object

In [82]:
df['emp_title'] = df['emp_title'].apply(clean_emp_title)

df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,...,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,int_rate_cleaned
0,20000,20000,20000.0,36 months,14.47,688.13,C,C2,Bus Driver,4 years,OWN,52000.0,Source Verified,Dec-2018,Charged Off,n,debt_consolidation,Debt consolidation,681xx,NE,30.65,1,Jun-1979,2,15.0,NaN,6,0,15048,73%,22,w,0.00,0.00,8169.130000,8169.13,5731.54,2437.59,0.0,0.0,0.0,Feb-2020,0.00,NaN,Mar-2020,0,NaN,1,Individual,NaN,...,4,14,4,6,NaN,0,0,2,95.0,50.0,0,0,61699,33157,20700,40999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,688.13,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,688.13,N,NaN,NaN,NaN,NaN,NaN,NaN,14.47
1,16000,16000,16000.0,36 months,14.47,550.51,C,C2,Carpenter,10+ years,RENT,150000.0,Source Verified,Dec-2018,Current,n,other,Other,917xx,CA,17.94,0,Nov-2001,1,NaN,NaN,24,0,43721,59.2%,29,w,9726.53,9726.53,8788.870000,8788.87,6273.47,2515.40,0.0,0.0,0.0,May-2020,0.00,Jun-2020,May-2020,0,NaN,1,Individual,NaN,...,22,25,19,24,0.0,0,0,0,100.0,85.7,0,0,116647,57465,32100,42847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,14.47
2,8425,8425,8425.0,36 months,27.27,345.18,E,E5,Senior Director Risk Management,3 years,MORTGAGE,450000.0,Verified,Dec-2018,Fully Paid,n,credit_card,Credit card refinancing,020xx,MA,12.37,0,Oct-1997,0,NaN,NaN,21,0,36812,65.7%,37,w,0.00,0.00,8735.148975,8735.15,8425.00,310.15,0.0,0.0,0.0,Feb-2019,8754.30,NaN,Mar-2020,0,NaN,1,Joint App,515000.0,...,13,17,8,21,0.0,0,0,2,100.0,50.0,0,0,768304,189194,45800,189054,141905.0,Jun-2013,0.0,1.0,4.0,77.4,1.0,4.0,0.0,0.0,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,27.27
3,10000,10000,10000.0,60 months,12.98,227.43,B,B5,Printer,9 years,RENT,60000.0,Not Verified,Dec-2018,Current,n,credit_card,Credit card refinancing,660xx,KS,14.90,0,May-2007,0,NaN,112.0,7,1,10677,54.2%,12,w,7787.21,7787.21,3805.020000,3805.02,2212.79,1592.23,0.0,0.0,0.0,May-2020,227.43,Jun-2020,May-2020,0,NaN,1,Individual,NaN,...,5,6,5,7,0.0,0,0,1,100.0,33.3,1,0,36200,27595,13000,16500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,12.98
4,30000,30000,30000.0,60 months,26.31,903.73,E,E4,Casino Host,10+ years,OWN,62000.0,Source Verified,Dec-2018,Charged Off,n,debt_consolidation,Debt consolidation,920xx,CA,37.24,0,Oct-2007,0,NaN,86.0,15,1,18890,39.8%,36,w,0.00,0

In [83]:
df['emp_title'].value_counts(dropna=False).head(20)

Unknown               20947
Teacher                2557
Manager                2395
Registered Nurse       1418
Driver                 1258
Supervisor             1160
Truck Driver            920
Rn                      834
Office Manager          805
Sales                   803
General Manager         791
Project Manager         720
Owner                   625
Director                523
Operations Manager      518
Sales Manager           500
Police Officer          440
Nurse                   425
Technician              420
Engineer                412
Name: emp_title, dtype: int64

In [84]:
df['emp_title'].nunique()

34902

In [85]:
df['emp_title'].isnull().sum()

0

In [86]:
[col for col in df if col.endswith('_d')]

['issue_d', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d']

In [87]:
df['issue_d'].head()

0    Dec-2018
1    Dec-2018
2    Dec-2018
3    Dec-2018
4    Dec-2018
Name: issue_d, dtype: object

In [88]:
df['issue_d'].tail()

128407    Oct-2018
128408    Oct-2018
128409    Oct-2018
128410    Oct-2018
128411    Oct-2018
Name: issue_d, dtype: object

In [0]:
df['issue_d'] = pd.to_datetime(df['issue_d'], infer_datetime_format=True)

In [90]:
df['issue_d'].head()


0   2018-12-01
1   2018-12-01
2   2018-12-01
3   2018-12-01
4   2018-12-01
Name: issue_d, dtype: datetime64[ns]

In [91]:
df['issue_d'][0]

Timestamp('2018-12-01 00:00:00')

In [92]:
df['issue_d'].dt.year

0         2018
1         2018
2         2018
3         2018
4         2018
          ... 
128407    2018
128408    2018
128409    2018
128410    2018
128411    2018
Name: issue_d, Length: 128412, dtype: int64

In [93]:
df['issue_d'].dt.month

0         12
1         12
2         12
3         12
4         12
          ..
128407    10
128408    10
128409    10
128410    10
128411    10
Name: issue_d, Length: 128412, dtype: int64

In [95]:
df['issue_year'] = df['issue_d'].dt.year
df['issue_month'] = df['issue_d'].dt.month

df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,...,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,int_rate_cleaned,issue_year,issue_month
0,20000,20000,20000.0,36 months,14.47,688.13,C,C2,Bus Driver,4 years,OWN,52000.0,Source Verified,2018-12-01,Charged Off,n,debt_consolidation,Debt consolidation,681xx,NE,30.65,1,Jun-1979,2,15.0,NaN,6,0,15048,73%,22,w,0.00,0.00,8169.130000,8169.13,5731.54,2437.59,0.0,0.0,0.0,Feb-2020,0.00,NaN,Mar-2020,0,NaN,1,Individual,NaN,...,4,6,NaN,0,0,2,95.0,50.0,0,0,61699,33157,20700,40999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,688.13,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,688.13,N,NaN,NaN,NaN,NaN,NaN,NaN,14.47,2018,12
1,16000,16000,16000.0,36 months,14.47,550.51,C,C2,Carpenter,10+ years,RENT,150000.0,Source Verified,2018-12-01,Current,n,other,Other,917xx,CA,17.94,0,Nov-2001,1,NaN,NaN,24,0,43721,59.2%,29,w,9726.53,9726.53,8788.870000,8788.87,6273.47,2515.40,0.0,0.0,0.0,May-2020,0.00,Jun-2020,May-2020,0,NaN,1,Individual,NaN,...,19,24,0.0,0,0,0,100.0,85.7,0,0,116647,57465,32100,42847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,14.47,2018,12
2,8425,8425,8425.0,36 months,27.27,345.18,E,E5,Senior Director Risk Management,3 years,MORTGAGE,450000.0,Verified,2018-12-01,Fully Paid,n,credit_card,Credit card refinancing,020xx,MA,12.37,0,Oct-1997,0,NaN,NaN,21,0,36812,65.7%,37,w,0.00,0.00,8735.148975,8735.15,8425.00,310.15,0.0,0.0,0.0,Feb-2019,8754.30,NaN,Mar-2020,0,NaN,1,Joint App,515000.0,...,8,21,0.0,0,0,2,100.0,50.0,0,0,768304,189194,45800,189054,141905.0,Jun-2013,0.0,1.0,4.0,77.4,1.0,4.0,0.0,0.0,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,27.27,2018,12
3,10000,10000,10000.0,60 months,12.98,227.43,B,B5,Printer,9 years,RENT,60000.0,Not Verified,2018-12-01,Current,n,credit_card,Credit card refinancing,660xx,KS,14.90,0,May-2007,0,NaN,112.0,7,1,10677,54.2%,12,w,7787.21,7787.21,3805.020000,3805.02,2212.79,1592.23,0.0,0.0,0.0,May-2020,227.43,Jun-2020,May-2020,0,NaN,1,Individual,NaN,...,5,7,0.0,0,0,1,100.0,33.3,1,0,36200,27595,13000,16500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,12.98,2018,12
4,30000,30000,30000.0,60 months,26.31,903.73,E,E4,Casino Host,10+ years,OWN,62000.0,Source Verified,2018-12-01,Charged Off,n,debt_consolidation,Debt consolidation,920xx,CA,37.24,0,Oct-2007,0,NaN,86.0,15,1,18890,3

In [0]:
df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'], infer_datetime_format=True)

In [97]:
df['earliest_cr_line'].head()

0   1979-06-01
1   2001-11-01
2   1997-10-01
3   2007-05-01
4   2007-10-01
Name: earliest_cr_line, dtype: datetime64[ns]

In [98]:
(df['issue_d'] - df['earliest_cr_line']).dt.days / 365

0         39.528767
1         17.093151
2         21.180822
3         11.594521
4         11.175342
            ...    
128407    33.189041
128408    36.276712
128409    12.342466
128410    10.005479
128411    12.090411
Length: 128412, dtype: float64

In [100]:
df['length_of_credit_history'] = (df['issue_d'] - df['earliest_cr_line']).dt.days / 365

df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,...,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,int_rate_cleaned,issue_year,issue_month,length_of_credit_history
0,20000,20000,20000.0,36 months,14.47,688.13,C,C2,Bus Driver,4 years,OWN,52000.0,Source Verified,2018-12-01,Charged Off,n,debt_consolidation,Debt consolidation,681xx,NE,30.65,1,1979-06-01,2,15.0,NaN,6,0,15048,73%,22,w,0.00,0.00,8169.130000,8169.13,5731.54,2437.59,0.0,0.0,0.0,Feb-2020,0.00,NaN,Mar-2020,0,NaN,1,Individual,NaN,...,6,NaN,0,0,2,95.0,50.0,0,0,61699,33157,20700,40999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,688.13,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,688.13,N,NaN,NaN,NaN,NaN,NaN,NaN,14.47,2018,12,39.528767
1,16000,16000,16000.0,36 months,14.47,550.51,C,C2,Carpenter,10+ years,RENT,150000.0,Source Verified,2018-12-01,Current,n,other,Other,917xx,CA,17.94,0,2001-11-01,1,NaN,NaN,24,0,43721,59.2%,29,w,9726.53,9726.53,8788.870000,8788.87,6273.47,2515.40,0.0,0.0,0.0,May-2020,0.00,Jun-2020,May-2020,0,NaN,1,Individual,NaN,...,24,0.0,0,0,0,100.0,85.7,0,0,116647,57465,32100,42847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,14.47,2018,12,17.093151
2,8425,8425,8425.0,36 months,27.27,345.18,E,E5,Senior Director Risk Management,3 years,MORTGAGE,450000.0,Verified,2018-12-01,Fully Paid,n,credit_card,Credit card refinancing,020xx,MA,12.37,0,1997-10-01,0,NaN,NaN,21,0,36812,65.7%,37,w,0.00,0.00,8735.148975,8735.15,8425.00,310.15,0.0,0.0,0.0,Feb-2019,8754.30,NaN,Mar-2020,0,NaN,1,Joint App,515000.0,...,21,0.0,0,0,2,100.0,50.0,0,0,768304,189194,45800,189054,141905.0,Jun-2013,0.0,1.0,4.0,77.4,1.0,4.0,0.0,0.0,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,27.27,2018,12,21.180822
3,10000,10000,10000.0,60 months,12.98,227.43,B,B5,Printer,9 years,RENT,60000.0,Not Verified,2018-12-01,Current,n,credit_card,Credit card refinancing,660xx,KS,14.90,0,2007-05-01,0,NaN,112.0,7,1,10677,54.2%,12,w,7787.21,7787.21,3805.020000,3805.02,2212.79,1592.23,0.0,0.0,0.0,May-2020,227.43,Jun-2020,May-2020,0,NaN,1,Individual,NaN,...,7,0.0,0,0,1,100.0,33.3,1,0,36200,27595,13000,16500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,12.98,2018,12,11.594521
4,30000,30000,30000.0,60 months,26.31,903.73,E,E4,Casino Host,10+ years,OWN,62000.0,Source Verified,2018-12-01,Charged Off,n,debt_consolidation,Debt consolidation,920xx

In [101]:
df['length_of_credit_history'].describe()

count    128412.000000
mean         16.054497
std           7.908317
min           3.084932
25%          11.093151
50%          14.427397
75%          19.846575
max          68.961644
Name: length_of_credit_history, dtype: float64

In [102]:
df['term'].head()

0     36 months
1     36 months
2     36 months
3     60 months
4     60 months
Name: term, dtype: object

In [0]:
def term_to_int(term):
    return int(term.strip(' months'))


df['term'] = df['term'].apply(term_to_int)

In [111]:
df['term'].dtypes

dtype('int64')

In [119]:
def loan_status(status):
    if status == 'Current':
        return '1'
    elif status == 'Fully Paid':
        return '0'

df['loan_status'] = df['loan_status'].apply(loan_status)

df['loan_status'].head()

0    None
1       1
2       0
3       1
4    None
Name: loan_status, dtype: object

In [120]:

df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,...,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,int_rate_cleaned,issue_year,issue_month,length_of_credit_history
0,20000,20000,20000.0,36,14.47,688.13,C,C2,Bus Driver,4 years,OWN,52000.0,Source Verified,2018-12-01,None,n,debt_consolidation,Debt consolidation,681xx,NE,30.65,1,1979-06-01,2,15.0,NaN,6,0,15048,73%,22,w,0.00,0.00,8169.130000,8169.13,5731.54,2437.59,0.0,0.0,0.0,Feb-2020,0.00,NaN,Mar-2020,0,NaN,1,Individual,NaN,...,6,NaN,0,0,2,95.0,50.0,0,0,61699,33157,20700,40999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,688.13,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,688.13,N,NaN,NaN,NaN,NaN,NaN,NaN,14.47,2018,12,39.528767
1,16000,16000,16000.0,36,14.47,550.51,C,C2,Carpenter,10+ years,RENT,150000.0,Source Verified,2018-12-01,1,n,other,Other,917xx,CA,17.94,0,2001-11-01,1,NaN,NaN,24,0,43721,59.2%,29,w,9726.53,9726.53,8788.870000,8788.87,6273.47,2515.40,0.0,0.0,0.0,May-2020,0.00,Jun-2020,May-2020,0,NaN,1,Individual,NaN,...,24,0.0,0,0,0,100.0,85.7,0,0,116647,57465,32100,42847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,14.47,2018,12,17.093151
2,8425,8425,8425.0,36,27.27,345.18,E,E5,Senior Director Risk Management,3 years,MORTGAGE,450000.0,Verified,2018-12-01,0,n,credit_card,Credit card refinancing,020xx,MA,12.37,0,1997-10-01,0,NaN,NaN,21,0,36812,65.7%,37,w,0.00,0.00,8735.148975,8735.15,8425.00,310.15,0.0,0.0,0.0,Feb-2019,8754.30,NaN,Mar-2020,0,NaN,1,Joint App,515000.0,...,21,0.0,0,0,2,100.0,50.0,0,0,768304,189194,45800,189054,141905.0,Jun-2013,0.0,1.0,4.0,77.4,1.0,4.0,0.0,0.0,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,27.27,2018,12,21.180822
3,10000,10000,10000.0,60,12.98,227.43,B,B5,Printer,9 years,RENT,60000.0,Not Verified,2018-12-01,1,n,credit_card,Credit card refinancing,660xx,KS,14.90,0,2007-05-01,0,NaN,112.0,7,1,10677,54.2%,12,w,7787.21,7787.21,3805.020000,3805.02,2212.79,1592.23,0.0,0.0,0.0,May-2020,227.43,Jun-2020,May-2020,0,NaN,1,Individual,NaN,...,7,0.0,0,0,1,100.0,33.3,1,0,36200,27595,13000,16500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,12.98,2018,12,11.594521
4,30000,30000,30000.0,60,26.31,903.73,E,E4,Casino Host,10+ years,OWN,62000.0,Source Verified,2018-12-01,None,n,debt_consolidation,Debt consolidation,920xx,CA,37.24,0,2007-10-01,0,NaN,86.0,15,1,18890,39.8%,36,w,0.00,0.00,2645

In [0]:
df['last_pymnt_d'] = pd.to_datetime(df['last_pymnt_d'], infer_datetime_format=True)

In [0]:
df['last_pymnt_d_month'] = df['last_pymnt_d'].dt.month
df['last_pymnt_d_year'] = df['last_pymnt_d'].dt.year

In [134]:
df['last_pymnt_d_month'].head()

0    2.0
1    5.0
2    2.0
3    5.0
4    4.0
Name: last_pymnt_d_month, dtype: float64

In [129]:
df['last_pymnt_d_year'].head()

0    2020.0
1    2020.0
2    2019.0
3    2020.0
4    2019.0
Name: last_pymnt_d_year, dtype: float64

# Stretch Goals

You can do more with the LendingClub or Instacart datasets.

LendingClub options:
- There's one other column in the dataframe with percent signs. Remove them and convert to floats. You'll need to handle missing values.
- Modify the `emp_title` column to replace titles with 'Other' if the title is not in the top 20. 
- Take initiatve and work on your own ideas!

Instacart options:
- Read [Instacart Market Basket Analysis, Winner's Interview: 2nd place, Kazuki Onodera](http://blog.kaggle.com/2017/09/21/instacart-market-basket-analysis-winners-interview-2nd-place-kazuki-onodera/), especially the **Feature Engineering** section. (Can you choose one feature from his bulleted lists, and try to engineer it with pandas code?)
- Read and replicate parts of [Simple Exploration Notebook - Instacart](https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-instacart). (It's the Python Notebook with the most upvotes for this Kaggle competition.)
- Take initiative and work on your own ideas!

You can uncomment and run the cells below to re-download and extract the Instacart data

In [0]:
# !wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

In [0]:
# !tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

In [0]:
# %cd instacart_2017_05_01